In [1]:
import pandas as pd
import sinta_scraper as sinta
import ast
from datetime import datetime
today = datetime.today()

#from bokeh.io import output_file, show
from bokeh.io import output_notebook, show, save
from bokeh.plotting import figure, ColumnDataSource, output_file, save
from bokeh.models import Band, Div, RangeSlider, Spinner, CategoricalColorMapper
from bokeh.palettes import d3

import numpy as np

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def bokeh_df(df):
    df['est_age'] = [today.year - (int(i[2:-4])-26) if i.startswith('11201') else today.year - (1900 + int(i[-4:-2])) for i in df.NIDN]
    df['areas'] = df['areas']
    df['scopus_doc'] = [ast.literal_eval(i)['documents'] for i in df['scopus']]
    df['scopus_cit'] = [ast.literal_eval(i)['citations'] for i in df['scopus']]
    df['scopus_ind'] = [ast.literal_eval(i)['h-index'] for i in df['scopus']]
    df['score_extracted'] = [ast.literal_eval(i)['overall'] for i in df['score']]
    df['score_3_years'] = [ast.literal_eval(i)['3_years'] for i in df['score']]
    df['score_extracted_v2'] = [ast.literal_eval(i)['overall_v2'] for i in df['score']]
    df['score_3_years_v2'] = [ast.literal_eval(i)['3_years_v2'] for i in df['score']]
    df_scopus = df.loc[:, ['est_age', 'scopus_doc', 'scopus_ind', 'name_x', 'NIDN', 'areas', 'score_extracted', 'score_3_years', 'score_extracted_v2', 'score_3_years_v2', 'Instansi']]
    return df_scopus

In [4]:
df1 = pd.read_csv('../tables/itb_bio_ready.csv', index_col=0, dtype=str)
df2 = pd.read_csv('../tables/ugm_bio_ready.csv', index_col=0, dtype=str)
df_ugm = bokeh_df(df2)
df_itb = bokeh_df(df1)

In [5]:
source_ugm = ColumnDataSource(data=df_ugm.reset_index())
source_itb = ColumnDataSource(data=df_itb.reset_index())

TOOLTIPS = [
    ("Score", "@score_extracted"),
    ("Name", "@name_x"),    
    ("(H-index, n-Articles)", "(@scopus_ind, @scopus_doc)"),
    ("Areas", "@areas"),  
    ("(Age, NIDN)", "(@est_age, @NIDN)"),
    ("Institusi", "@Instansi"),
    ("(x,y)", "($x, $y)")
]

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p = figure(plot_width=600, plot_height=600, tooltips=TOOLTIPS,
           title="Life Sciences SINTA", x_axis_label="Estimated Age",
           y_axis_label="SINTA Score")

# use whatever palette you want...
#palette = d3['Category10'][len(df['Instansi'].unique())]
palette= ['#1f77b4', '#ff7f0e']
#color_map = CategoricalColorMapper(factors=df1['Instansi'].unique(),
#                                   palette=palette)


p.scatter('est_age', 'score_extracted', size=15, fill_alpha=0.5, source=source_ugm,
         #color={'field': 'Instansi', 'transform': color_map}, legend_label='UGM')
          color='#1f77b4', legend_label='UGM')

p.scatter('est_age', 'score_extracted', size=15, fill_alpha=0.5, source=source_itb,
         color='#ff7f0e', legend_label='ITB')

p.legend.location = "top_left"
p.legend.click_policy="hide"

show(p)

# set output to static HTML file
output_file(filename="../figures/bio_sinta.html", title="Static HTML file")
save(p)

'/home/matinnu/others/SINTA_data_mining/figures/bio_sinta.html'